## Preprocessing

In [2]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("../Resources/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [3]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(columns=["EIN", "NAME"], axis=1)
application_df.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [4]:
# Determine the number of unique values in each column.
application_df.nunique()

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [5]:
# Look at APPLICATION_TYPE value counts for binning
application_df["APPLICATION_TYPE"].value_counts()

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T15        2
T29        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [6]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
# I'm choosing to replace application types that have a count of less than 100
app_counts = application_df["APPLICATION_TYPE"].value_counts()

application_types_to_replace = []

for index,value in app_counts.items():
    if value < 100:
        application_types_to_replace.append(index)

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
T9         156
Other      120
Name: APPLICATION_TYPE, dtype: int64

In [7]:
# Look at CLASSIFICATION value counts for binning
application_df["CLASSIFICATION"].value_counts()

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C1728        1
C2150        1
C6100        1
C2570        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [9]:
# You may find it helpful to look at CLASSIFICATION value counts >1
class_values = application_df["CLASSIFICATION"].value_counts()

print(class_values[class_values > 1])
        


C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C2300       32
C7200       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1278       10
C1238       10
C1235        9
C1237        9
C7210        7
C1720        6
C4100        6
C2400        6
C1600        5
C1257        5
C2710        3
C0           3
C1260        3
C1246        2
C3200        2
C1267        2
C1256        2
C1234        2
Name: CLASSIFICATION, dtype: int64


In [10]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
# I'm choosing any classification with a value less than 100 to be replaced
classifications_to_replace = []

for index,value in class_values.items():
    if value < 100:
        classifications_to_replace.append(index)

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")
    
# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
Other      669
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
Name: CLASSIFICATION, dtype: int64

In [11]:
# Convert categorical data to numeric with `pd.get_dummies`
application_df = pd.get_dummies(application_df)

In [12]:
# Split our preprocessed data into our features and target arrays
y = application_df["IS_SUCCESSFUL"].values
X = application_df.drop(columns="IS_SUCCESSFUL").values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [13]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Add Additional Layers with more Neurons to try and Optimize the Model

In [14]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train[0])
hidden_nodes_layer1 = 12
hidden_nodes_layer2 = 10
hidden_nodes_layer3 = 8
hidden_nodes_layer4 = 4

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Third hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="relu"))

# Fourth hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer4, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 12)                612       
                                                                 
 dense_1 (Dense)             (None, 10)                130       
                                                                 
 dense_2 (Dense)             (None, 8)                 88        
                                                                 
 dense_3 (Dense)             (None, 4)                 36        
                                                                 
 dense_4 (Dense)             (None, 1)                 5         
                                                                 
Total params: 871
Trainable params: 871
Non-trainable params: 0
_________________________________________________________________


In [15]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [16]:
# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=150)

Epoch 1/150
804/804 [==============================] - 3s 2ms/step - loss: 0.5966 - accuracy: 0.7075
Epoch 2/150
804/804 [==============================] - 1s 2ms/step - loss: 0.5611 - accuracy: 0.7271
Epoch 3/150
804/804 [==============================] - 1s 2ms/step - loss: 0.5553 - accuracy: 0.7281
Epoch 4/150
804/804 [==============================] - 1s 2ms/step - loss: 0.5524 - accuracy: 0.7300
Epoch 5/150
804/804 [==============================] - 1s 2ms/step - loss: 0.5508 - accuracy: 0.7302
Epoch 6/150
804/804 [==============================] - 1s 2ms/step - loss: 0.5497 - accuracy: 0.7308
Epoch 7/150
804/804 [==============================] - 1s 2ms/step - loss: 0.5486 - accuracy: 0.7311
Epoch 8/150
804/804 [==============================] - 1s 2ms/step - loss: 0.5478 - accuracy: 0.7311
Epoch 9/150
804/804 [==============================] - 1s 2ms/step - loss: 0.5472 - accuracy: 0.7318
Epoch 10/150
804/804 [==============================] - 1s 2ms/step - loss: 0.5470 - accura

804/804 [==============================] - 1s 2ms/step - loss: 0.5393 - accuracy: 0.7350
Epoch 82/150
804/804 [==============================] - 1s 2ms/step - loss: 0.5388 - accuracy: 0.7368
Epoch 83/150
804/804 [==============================] - 1s 2ms/step - loss: 0.5390 - accuracy: 0.7363
Epoch 84/150
804/804 [==============================] - 1s 2ms/step - loss: 0.5388 - accuracy: 0.7365
Epoch 85/150
804/804 [==============================] - 1s 2ms/step - loss: 0.5390 - accuracy: 0.7364
Epoch 86/150
804/804 [==============================] - 1s 2ms/step - loss: 0.5386 - accuracy: 0.7372
Epoch 87/150
804/804 [==============================] - 1s 2ms/step - loss: 0.5389 - accuracy: 0.7362
Epoch 88/150
804/804 [==============================] - 1s 2ms/step - loss: 0.5391 - accuracy: 0.7365
Epoch 89/150
804/804 [==============================] - 1s 2ms/step - loss: 0.5389 - accuracy: 0.7356
Epoch 90/150
804/804 [==============================] - 1s 2ms/step - loss: 0.5386 - accuracy: 

In [17]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.5553 - accuracy: 0.7280 - 526ms/epoch - 2ms/step
Loss: 0.555252730846405, Accuracy: 0.7280466556549072


## Changing the Activation Functions of the Hidden Layers to Try and Optimize the Model

In [18]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train[0])
hidden_nodes_layer1 = 12
hidden_nodes_layer2 = 10

nn2 = tf.keras.models.Sequential()

# First hidden layer
nn2.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="tanh")
)

# Second hidden layer
nn2.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="tanh"))

# Output layer
nn2.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn2.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_5 (Dense)             (None, 12)                612       
                                                                 
 dense_6 (Dense)             (None, 10)                130       
                                                                 
 dense_7 (Dense)             (None, 1)                 11        
                                                                 
Total params: 753
Trainable params: 753
Non-trainable params: 0
_________________________________________________________________


In [19]:
# Compile the model
nn2.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [20]:
# Train the model
fit_model = nn2.fit(X_train_scaled, y_train, epochs=150)

Epoch 1/150
804/804 [==============================] - 2s 2ms/step - loss: 0.5964 - accuracy: 0.6980
Epoch 2/150
804/804 [==============================] - 1s 2ms/step - loss: 0.5621 - accuracy: 0.7257
Epoch 3/150
804/804 [==============================] - 1s 2ms/step - loss: 0.5568 - accuracy: 0.7280
Epoch 4/150
804/804 [==============================] - 1s 2ms/step - loss: 0.5539 - accuracy: 0.7293
Epoch 5/150
804/804 [==============================] - 1s 2ms/step - loss: 0.5524 - accuracy: 0.7287
Epoch 6/150
804/804 [==============================] - 1s 2ms/step - loss: 0.5511 - accuracy: 0.7299
Epoch 7/150
804/804 [==============================] - 1s 2ms/step - loss: 0.5503 - accuracy: 0.7302
Epoch 8/150
804/804 [==============================] - 1s 2ms/step - loss: 0.5492 - accuracy: 0.7304
Epoch 9/150
804/804 [==============================] - 1s 2ms/step - loss: 0.5487 - accuracy: 0.7313
Epoch 10/150
804/804 [==============================] - 1s 2ms/step - loss: 0.5475 - accura

804/804 [==============================] - 1s 2ms/step - loss: 0.5373 - accuracy: 0.7369
Epoch 82/150
804/804 [==============================] - 1s 2ms/step - loss: 0.5370 - accuracy: 0.7381
Epoch 83/150
804/804 [==============================] - 1s 2ms/step - loss: 0.5369 - accuracy: 0.7383
Epoch 84/150
804/804 [==============================] - 1s 2ms/step - loss: 0.5371 - accuracy: 0.7382
Epoch 85/150
804/804 [==============================] - 1s 2ms/step - loss: 0.5368 - accuracy: 0.7369
Epoch 86/150
804/804 [==============================] - 1s 2ms/step - loss: 0.5370 - accuracy: 0.7382
Epoch 87/150
804/804 [==============================] - 1s 2ms/step - loss: 0.5369 - accuracy: 0.7372
Epoch 88/150
804/804 [==============================] - 1s 2ms/step - loss: 0.5370 - accuracy: 0.7382
Epoch 89/150
804/804 [==============================] - 1s 2ms/step - loss: 0.5368 - accuracy: 0.7375
Epoch 90/150
804/804 [==============================] - 1s 2ms/step - loss: 0.5368 - accuracy: 

In [21]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn2.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.5536 - accuracy: 0.7298 - 563ms/epoch - 2ms/step
Loss: 0.5536264777183533, Accuracy: 0.7297959327697754


## Using a Combination of More Hidden Layers with Different Activation Functions As Well As More Epochs for Training

In [22]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train[0])
hidden_nodes_layer1 = 12
hidden_nodes_layer2 = 10
hidden_nodes_layer3 = 8
hidden_nodes_layer4 = 4

nn3 = tf.keras.models.Sequential()

# First hidden layer
nn3.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="sigmoid")
)

# Second hidden layer
nn3.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="sigmoid"))

# Third hidden layer
nn3.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="sigmoid"))

# Fourth hidden layer
nn3.add(tf.keras.layers.Dense(units=hidden_nodes_layer4, activation="sigmoid"))

# Output layer
nn3.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn3.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_8 (Dense)             (None, 12)                612       
                                                                 
 dense_9 (Dense)             (None, 10)                130       
                                                                 
 dense_10 (Dense)            (None, 8)                 88        
                                                                 
 dense_11 (Dense)            (None, 4)                 36        
                                                                 
 dense_12 (Dense)            (None, 1)                 5         
                                                                 
Total params: 871
Trainable params: 871
Non-trainable params: 0
_________________________________________________________________


In [23]:
# Compile the model
nn3.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [24]:
# Train the model
fit_model3 = nn3.fit(X_train_scaled, y_train, epochs=200)

Epoch 1/200
804/804 [==============================] - 3s 2ms/step - loss: 0.6603 - accuracy: 0.6275
Epoch 2/200
804/804 [==============================] - 1s 2ms/step - loss: 0.5808 - accuracy: 0.7288
Epoch 3/200
804/804 [==============================] - 1s 2ms/step - loss: 0.5747 - accuracy: 0.7311
Epoch 4/200
804/804 [==============================] - 1s 2ms/step - loss: 0.5733 - accuracy: 0.7304
Epoch 5/200
804/804 [==============================] - 1s 2ms/step - loss: 0.5727 - accuracy: 0.7311
Epoch 6/200
804/804 [==============================] - 1s 2ms/step - loss: 0.5724 - accuracy: 0.7304
Epoch 7/200
804/804 [==============================] - 1s 2ms/step - loss: 0.5715 - accuracy: 0.7310
Epoch 8/200
804/804 [==============================] - 1s 2ms/step - loss: 0.5708 - accuracy: 0.7313
Epoch 9/200
804/804 [==============================] - 1s 2ms/step - loss: 0.5695 - accuracy: 0.7310
Epoch 10/200
804/804 [==============================] - 1s 2ms/step - loss: 0.5677 - accura

804/804 [==============================] - 1s 2ms/step - loss: 0.5417 - accuracy: 0.7359
Epoch 82/200
804/804 [==============================] - 1s 2ms/step - loss: 0.5422 - accuracy: 0.7358
Epoch 83/200
804/804 [==============================] - 1s 2ms/step - loss: 0.5417 - accuracy: 0.7349
Epoch 84/200
804/804 [==============================] - 1s 2ms/step - loss: 0.5418 - accuracy: 0.7364
Epoch 85/200
804/804 [==============================] - 1s 2ms/step - loss: 0.5417 - accuracy: 0.7350
Epoch 86/200
804/804 [==============================] - 1s 2ms/step - loss: 0.5417 - accuracy: 0.7355
Epoch 87/200
804/804 [==============================] - 1s 2ms/step - loss: 0.5413 - accuracy: 0.7350
Epoch 88/200
804/804 [==============================] - 1s 2ms/step - loss: 0.5414 - accuracy: 0.7346
Epoch 89/200
804/804 [==============================] - 1s 2ms/step - loss: 0.5413 - accuracy: 0.7354
Epoch 90/200
804/804 [==============================] - 1s 2ms/step - loss: 0.5414 - accuracy: 

804/804 [==============================] - 1s 2ms/step - loss: 0.5388 - accuracy: 0.7372
Epoch 161/200
804/804 [==============================] - 1s 2ms/step - loss: 0.5386 - accuracy: 0.7368
Epoch 162/200
804/804 [==============================] - 1s 2ms/step - loss: 0.5387 - accuracy: 0.7378
Epoch 163/200
804/804 [==============================] - 1s 2ms/step - loss: 0.5385 - accuracy: 0.7374
Epoch 164/200
804/804 [==============================] - 1s 2ms/step - loss: 0.5387 - accuracy: 0.7375
Epoch 165/200
804/804 [==============================] - 1s 2ms/step - loss: 0.5386 - accuracy: 0.7378
Epoch 166/200
804/804 [==============================] - 1s 2ms/step - loss: 0.5385 - accuracy: 0.7375
Epoch 167/200
804/804 [==============================] - 1s 2ms/step - loss: 0.5389 - accuracy: 0.7365
Epoch 168/200
804/804 [==============================] - 1s 2ms/step - loss: 0.5387 - accuracy: 0.7372
Epoch 169/200
804/804 [==============================] - 1s 2ms/step - loss: 0.5383 - a

In [25]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn3.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5516 - accuracy: 0.7313 - 479ms/epoch - 2ms/step
Loss: 0.5516342520713806, Accuracy: 0.7313119769096375


## Changing Data Preparation to Try and Optimize the Model

In [26]:
application_df2 = pd.read_csv("../Resources/charity_data.csv")
application_df2.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


### Add the columns 'STATUS' and 'SPECIAL_CONSIDERATIONS' to the list of columns to be dropped

In [28]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'. Also drop the 'STATUS' and 'SPECIAL_CONSIDERATIONS' columns to try
# and prevent confusing the model
application_df2 = application_df2.drop(columns=["EIN", "NAME", "STATUS", "SPECIAL_CONSIDERATIONS"], axis=1)
application_df2.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,INCOME_AMT,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,0,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1-9999,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,0,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,10000-24999,6692,1
4,T3,Independent,C1000,Heathcare,Trust,100000-499999,142590,1


### Change the number of bins for Application Type

In [30]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
# I'm choosing to replace application types that have a count of less than 700
app_counts2 = application_df2["APPLICATION_TYPE"].value_counts()

application_types_to_replace2 = []

for index,value in app_counts2.items():
    if value < 700:
        application_types_to_replace2.append(index)

# Replace in dataframe
for app in application_types_to_replace2:
    application_df2['APPLICATION_TYPE'] = application_df2['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df2['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
Other      804
T8         737
T7         725
Name: APPLICATION_TYPE, dtype: int64

### Change the number of bins for Classification

In [32]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
# I'm choosing any classification with a value less than 100 to be replaced
classifications_to_replace2 = []

for index,value in class_values.items():
    if value < 700:
        classifications_to_replace2.append(index)

# Replace in dataframe
for cls in classifications_to_replace2:
    application_df2['CLASSIFICATION'] = application_df2['CLASSIFICATION'].replace(cls,"Other")
    
# Check to make sure binning was successful
application_df2['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
Other     1484
C7000      777
Name: CLASSIFICATION, dtype: int64

In [33]:
# Convert categorical data to numeric with `pd.get_dummies`
application_df2 = pd.get_dummies(application_df2)

In [42]:
# Split our preprocessed data into our features and target arrays
y = application_df2["IS_SUCCESSFUL"].values
X = application_df2.drop(columns="IS_SUCCESSFUL").values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [43]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [44]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train[0])
hidden_nodes_layer1 = 12
hidden_nodes_layer2 = 10
hidden_nodes_layer3 = 8
hidden_nodes_layer4 = 4

nn4 = tf.keras.models.Sequential()

# First hidden layer
nn4.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn4.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Third hidden layer
nn4.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="relu"))

# Fourth hidden layer
nn4.add(tf.keras.layers.Dense(units=hidden_nodes_layer4, activation="relu"))

# Output layer
nn4.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn4.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_23 (Dense)            (None, 12)                492       
                                                                 
 dense_24 (Dense)            (None, 10)                130       
                                                                 
 dense_25 (Dense)            (None, 8)                 88        
                                                                 
 dense_26 (Dense)            (None, 4)                 36        
                                                                 
 dense_27 (Dense)            (None, 1)                 5         
                                                                 
Total params: 751
Trainable params: 751
Non-trainable params: 0
_________________________________________________________________


In [45]:
# Compile the model
nn4.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [46]:
# Train the model
fit_model = nn4.fit(X_train_scaled, y_train, epochs=150)

Epoch 1/150
804/804 [==============================] - 3s 2ms/step - loss: 0.5953 - accuracy: 0.6964
Epoch 2/150
804/804 [==============================] - 1s 2ms/step - loss: 0.5589 - accuracy: 0.7280
Epoch 3/150
804/804 [==============================] - 1s 2ms/step - loss: 0.5543 - accuracy: 0.7280
Epoch 4/150
804/804 [==============================] - 1s 2ms/step - loss: 0.5521 - accuracy: 0.7293
Epoch 5/150
804/804 [==============================] - 1s 2ms/step - loss: 0.5501 - accuracy: 0.7309
Epoch 6/150
804/804 [==============================] - 1s 2ms/step - loss: 0.5490 - accuracy: 0.7290
Epoch 7/150
804/804 [==============================] - 1s 2ms/step - loss: 0.5482 - accuracy: 0.7299
Epoch 8/150
804/804 [==============================] - 1s 2ms/step - loss: 0.5475 - accuracy: 0.7311
Epoch 9/150
804/804 [==============================] - 1s 2ms/step - loss: 0.5470 - accuracy: 0.7309
Epoch 10/150
804/804 [==============================] - 1s 2ms/step - loss: 0.5466 - accura

804/804 [==============================] - 1s 2ms/step - loss: 0.5393 - accuracy: 0.7363
Epoch 82/150
804/804 [==============================] - 1s 2ms/step - loss: 0.5389 - accuracy: 0.7377
Epoch 83/150
804/804 [==============================] - 1s 2ms/step - loss: 0.5389 - accuracy: 0.7367
Epoch 84/150
804/804 [==============================] - 1s 2ms/step - loss: 0.5387 - accuracy: 0.7362
Epoch 85/150
804/804 [==============================] - 1s 2ms/step - loss: 0.5386 - accuracy: 0.7371
Epoch 86/150
804/804 [==============================] - 1s 2ms/step - loss: 0.5387 - accuracy: 0.7355
Epoch 87/150
804/804 [==============================] - 1s 2ms/step - loss: 0.5388 - accuracy: 0.7352
Epoch 88/150
804/804 [==============================] - 1s 2ms/step - loss: 0.5385 - accuracy: 0.7374
Epoch 89/150
804/804 [==============================] - 1s 2ms/step - loss: 0.5389 - accuracy: 0.7365
Epoch 90/150
804/804 [==============================] - 1s 2ms/step - loss: 0.5388 - accuracy: 

In [47]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn4.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.5537 - accuracy: 0.7254 - 558ms/epoch - 2ms/step
Loss: 0.553717315196991, Accuracy: 0.7253644466400146


In [48]:
# Export our model to HDF5 file
nn4.save("AlphabetSoupCharity_Optimization.h5")